In [1]:
import pandas as pd
import numpy as np
import datetime
import random
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import ExtraTreesRegressor

rs = 420
random.seed(rs)

print(datetime.datetime.today().strftime("%HH:%MM %dd"))

featuers = ['vesselId', 'hour', 'day', 'month', 'minute', 'vesselType', 'yearBuilt', 'length', 'breadth', 'CEU', 'DWT', 'GT', 'portLatitude', 'portLongitude', 'hour_sch', 'day_sch', 'month_sch', 'minute_sch', 'lat_shift_1', 'lon_shift_1', 'minute_shift_1', 'hour_shift_1', 'day_shift_1', 'month_shift_1', 'cog_1', 'heading_1', 'lat_shift_2', 'lon_shift_2', 'minute_shift_2', 'hour_shift_2', 'day_shift_2', 'month_shift_2', 'cog_2', 'heading_2', 'lat_shift_3', 'lon_shift_3', 'minute_shift_3', 'hour_shift_3', 'day_shift_3', 'month_shift_3', 'cog_3', 'heading_3', 'lat_shift_4', 'lon_shift_4', 'minute_shift_4', 'hour_shift_4', 'day_shift_4', 'month_shift_4', 'cog_4', 'heading_4', 'lat_shift_5', 'lon_shift_5', 'minute_shift_5', 'hour_shift_5', 'day_shift_5', 'month_shift_5', 'cog_5', 'heading_5', 'cog', 'heading', 'time_diff_1', 'time_diff_2', 'time_diff_3', 'time_diff_4', 'time_diff_5', 'time_diff']
vessel_encoder = LabelEncoder()
timesteps = 5
# test-test split
liste = []
for i in range(len(featuers)):
    liste.append(pd.read_csv('x_data_'+str(i)+'.csv', sep=','))

featuers = ['vesselId', 'lat_shift_1', 'lon_shift_1', 'lat_shift_2', 'lon_shift_2', 'lat_shift_3', 'lon_shift_3', 'time_diff', 'time_diff_1', 'time_diff_2']
X = liste[0].join(liste[1:])
X['vesselId'] = vessel_encoder.fit_transform(X['vesselId'])
X = X[featuers]
print(X.tail(1))
y_1 = pd.read_csv('y_data_1.csv', sep=',')
y_2 = pd.read_csv('y_data_2.csv', sep=',')
y = y_2.join(y_1)

X_0 = pd.read_csv('x_test.csv', sep=',')
X_1 = pd.read_csv('x_test_1.csv', sep=',')
X_2 = pd.read_csv('x_test_2.csv', sep=',')
X_3 = pd.read_csv('x_test_3.csv', sep=',')
X_4 = pd.read_csv('x_test_4.csv', sep=',')
X_test = X_0.join([X_1, X_2, X_3, X_4])
X_test = X_test[featuers]

model = ExtraTreesRegressor(n_estimators=180, random_state=rs)
scaler = StandardScaler()
X_train = scaler.fit_transform(X)
# Train a RandomForest model
model.fit(X_train, y)


def sjekk(liste):
        nyliste = [0, 0]
        flag = 0
        #print('longitude error:', liste[0])
        if(liste[0]<-180):
            nyliste[0] = 180+(liste[0]+180)
        elif(liste[0]>180):
            nyliste[0] = -180+(liste[0]-180)
        else:
            flag = 1
            nyliste[0] = liste[0]

        if(liste[1]<-90):
            nyliste[1] = -90-(liste[1]+90)
        elif(liste[1]>90):
            nyliste[1] = 90-(liste[1]-90)
        else:
            nyliste[1] = liste[1]
            flag = 1
        
        if(flag==0):
            print("Value exceided lat lon")

        return nyliste

results_with_index = []
dictlast = {}
    # Group by vesselId to process each vessel's data independently
for idx, row in X_test.iterrows():
    # Prepare test data for this vessel and retain original index
    row.to_frame()
    print(row)
    if row['vesselId'] in dictlast:
        row['lat_shift_3'] = row['lat_shift_2'] 
        row['lon_shift_3'] = row['lon_shift_2']
        row['lat_shift_2'] = row['lat_shift_1']
        row['lon_shift_2'] = row['lon_shift_1'] 
        row['lat_shift_1'] = dictlast[row['vesselId']][1]
        row['lon_shift_1'] = dictlast[row['vesselId']][0]
        
        
    result = model.predict(row.array.reshape(1, -1))
    print(result)
    result = sjekk(result[0])
    dictlast[row['vesselId']] = result
    results_with_index.append([idx, result[0], result[1]])
    #print(results_with_index)

# Concatenate results and sort by original order of ais_test
final_predictions = pd.DataFrame(results_with_index, columns=['ID', 'longitude_predicted', 'latitude_predicted'])
print(len(final_predictions['ID']))
print(final_predictions)
print("Latitude: max, min", final_predictions['latitude_predicted'].max(), final_predictions['latitude_predicted'].min())
print("Longitude: max, min", final_predictions['longitude_predicted'].max(), final_predictions['longitude_predicted'].min())
# Save the predictions
final_predictions.to_csv('ais_test_predictions_6.csv', index=False)
print(datetime.datetime.today().strftime("%HH:%MM %dd"))



20H:33M 24d
p6:          vesselId  hour  day  month  minute  vesselType  length  breadth   
1522060       682    23    7      5      59        83.0  182.00     25.0  \
1522061        85    23    7      5      59        83.0  199.00     32.0   
1522062       459    23    7      5      59        83.0  199.96     38.0   
1522063       596    23    7      5      59        21.0  186.00     27.0   
1522064       637    23    7      5      59        83.0  200.00     32.0   

          CEU      DWT  ...  lon_shift_4  lat_shift_5  lon_shift_5    cog   
1522060   300  12502.0  ...     -5.84482     51.09372     -5.84536  359.1  \
1522061  6354  22160.0  ...    -12.10865     38.47765    -12.13535   12.3   
1522062  7429  18241.0  ...     -4.66703     49.77723     -4.50058  269.8   
1522063  1400   7150.0  ...     10.29855     37.64190     11.32413    8.0   
1522064  6215  18907.0  ...    -74.95443     38.70218    -74.90653  336.0   

         heading  time_diff_1  time_diff_2  time_diff_3  time_di